In [1]:
import pandas as pd
import numpy as np
# import trading_strategy_utils as tsu
import update_dataframe as ud
import grid_search as gs

In [2]:
# Load the eth data
eth_df = pd.read_csv('data/ETH.csv', parse_dates=['ts_hour'])

# Select March in 2018
eth_df_march_18 = eth_df[(eth_df['ts_hour'].dt.month == 3) & (eth_df['ts_hour'].dt.year == 2018)]
eth_df_march_18 = eth_df_march_18[['ts_hour', 'pnl']]

eth_df_march_18.head()

,ts_hour,pnl
1382,2018-03-01 00:00:00,0.000713
1383,2018-03-01 01:00:00,0.003457
1384,2018-03-01 02:00:00,0.001399
1385,2018-03-01 03:00:00,-0.000612
1386,2018-03-01 04:00:00,0.004522


In [3]:
eth_df_march_18.tail()

,ts_hour,pnl
2121,2018-03-31 19:00:00,-0.007341
2122,2018-03-31 20:00:00,-0.014163
2123,2018-03-31 21:00:00,0.003250
2124,2018-03-31 22:00:00,0.005695
2125,2018-03-31 23:00:00,-0.003175


### Grid Search for EMA(Short & Long) params

In [4]:
# Define the range of parameters for grid search
short_window_range = range(4, 7, 1)
long_window_range = range(11, 15, 1)


# Perform grid search
grid_search_march_18 = gs.grid_search_2_params(eth_df_march_18, short_window_range, long_window_range)
grid_search_march_18_sorted = grid_search_march_18.sort_values('total_weighted_pnl', ascending=False)

short_window: 4, long_window: 11, total_weighted_pnl: 0.19894079922294572
short_window: 4, long_window: 12, total_weighted_pnl: 0.22448681198322112
short_window: 4, long_window: 13, total_weighted_pnl: 0.2105050693966634
short_window: 4, long_window: 14, total_weighted_pnl: 0.2028818443796852
short_window: 5, long_window: 11, total_weighted_pnl: 0.20503926123176158
short_window: 5, long_window: 12, total_weighted_pnl: 0.22844701442848622
short_window: 5, long_window: 13, total_weighted_pnl: 0.2144988020902957
short_window: 5, long_window: 14, total_weighted_pnl: 0.2120435671401853
short_window: 6, long_window: 11, total_weighted_pnl: 0.21830476621637168
short_window: 6, long_window: 12, total_weighted_pnl: 0.21168343054162403
short_window: 6, long_window: 13, total_weighted_pnl: 0.20846198640630875
short_window: 6, long_window: 14, total_weighted_pnl: 0.22979121972228195
Grid search finished.


In [5]:
grid_search_march_18_sorted

,short_window,long_window,total_weighted_pnl
11,6,14,0.229791
5,5,12,0.228447
1,4,12,0.224487
8,6,11,0.218305
6,5,13,0.214499
7,5,14,0.212044
9,6,12,0.211683
2,4,13,0.210505
10,6,13,0.208462
4,5,11,0.205039


# Run the trading strategy with the best EMA(Short & Long) params found by grid search

In [6]:
best_short_window = grid_search_march_18.iloc[0]['short_window']
best_long_window = grid_search_march_18.iloc[0]['long_window']
# best_short_window = 6
# best_long_window = 13
true_weight = 1

print('Best short window:', best_short_window)
print('Best long window:', best_long_window)
# Create a new DataFrame to store results with the best parameters
strategy_result = pd.DataFrame(columns=['ts_hour', 'pnl', 'EMA_short', 'EMA_long', 'is_positive', 'signal', 'count', 'consistency', 'is_continuous', 'class', 'weight_percentage', 'weight', 'weighted_pnl'])
results_list = []

# Update the DataFrame with the best parameters
for _, row in eth_df_march_18.iterrows():
    strategy_result = ud.update_dataframe_with_new_row(strategy_result, row, results_list, short_window=best_short_window, long_window=best_long_window, true_weight=true_weight)
print("Done")

Best short window: 4.0
Best long window: 11.0
Done


In [7]:
strategy_result[10:30].head(10)

,ts_hour,pnl,EMA_short,EMA_long,is_positive,signal,count,consistency,is_continuous,class,weight_percentage,weight,weighted_pnl
10,2018-03-01 10:00:00,0.000386,0.000275,0.000541,1,0,0,NaN,0,2.3,20,0.2,0.0
11,2018-03-01 11:00:00,0.000006,0.000167,0.000452,1,0,0,NaN,0,2.3,20,0.2,0.000001
12,2018-03-01 12:00:00,-0.001158,-0.000363,0.000184,0,0,0,NaN,0,0,0,0.0,-0.000232
13,2018-03-01 13:00:00,0.001554,0.000404,0.000412,1,0,0,NaN,0,2.3,20,0.2,0.0
14,2018-03-01 14:00:00,-0.000050,0.000222,0.000335,0,0,0,NaN,0,0,0,0.0,-0.00001
15,2018-03-01 15:00:00,-0.003276,-0.001177,-0.000267,0,0,0,NaN,0,0,0,0.0,-0.0
16,2018-03-01 16:00:00,0.002447,0.000272,0.000185,1,1,1,"[15, 16]",1,1,100,1.0,0.0
17,2018-03-01 17:00:00,0.000454,0.000345,0.000230,1,0,-1,"[15, 16, 17]",1,2.2,20,0.2,0.000454
18,2018-03-01 18:00:00,0.002197,0.001086,0.000558,1,0,0,NaN,0,2.3,20,0.2,0.000439
19,2018-03-01 19:00:00,-0.002221,-0.000237,0.000095,0,-1,0,NaN,0,0,0,0.0,-0.000444


In [8]:
strategy_result['class'].value_counts()

class
0      302
2.3    256
1      104
2.2     68
3.1     14
Name: count, dtype: int64

In [9]:
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
import nbformat

from plotly.subplots import make_subplots

In [ ]:
# Create the cumulative weighted PnL column
strategy_result_copy = strategy_result.copy()
strategy_result_copy['cumulative_weighted_pnl'] = strategy_result_copy['weighted_pnl'].cumsum()


In [15]:
# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)


# Show the plot
fig.show()

In [16]:
# Define shape and color map for different classes
class_marker_map = {
    '1': {'color': 'red', 'symbol': 'circle', 'size': 10},
    '2.1': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.2': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.3': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '3.1': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.2': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.3': {'color': 'green', 'symbol': 'square', 'size': 6}
}

# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Add Class and Weight points to the plot
for class_type in strategy_result_copy['class'].unique():
    if class_type not in class_marker_map:
        continue
    class_df = strategy_result_copy[strategy_result_copy['class'] == class_type]
    marker = class_marker_map[str(class_type)]
    fig.add_trace(go.Scatter(
        x=class_df['ts_hour'],
        y=class_df['cumulative_weighted_pnl'],
        mode='markers',
        name=f'Class {class_type}',
        marker=dict(color=marker['color'], symbol=marker['symbol'], size=marker['size']),
        text=[f"Class: {row['class']}<br>Weight: {row['weight']:.2f}" for index, row in class_df.iterrows()],
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)

# Show the plot
fig.show()

In [11]:
strategy_result.weighted_pnl.sum()

0.19894079922294572